In [125]:
import pandas as pd
import numpy as np

In [126]:
df = pd.read_excel('housesTRIAL.xlsx')
df.drop_duplicates(inplace=True)
df.head()

,Price (HUF),Size,Rooms,Address,Link
0,120 000,35,"1 living room(s), 1 bedroom(s), 1 bathroom(s)","Debrecen, Sinai Miklós",https://www.findahome.hu/ingatlanok/debrecen-s...
1,140 000,59,"1 living room(s), 1 bedroom(s), 1 bathroom(s)","Debrecen, Kardos Albert",https://www.findahome.hu/ingatlanok/debrecen-k...
2,280 000,100,"1 living room(s), 4 bedroom(s), 1 bathroom(s)","Debrecen, Komlóssy street",https://www.findahome.hu/ingatlanok/debrecen1-...
3,420 000,112,"1 living room(s), 3 bedroom(s), 1 bathroom(s)","Debrecen, Rákóczi",https://www.findahome.hu/ingatlanok/debrecen-b...
4,180 000,56,"0 living room(s), 2 bedroom(s), 1 bathroom(s)","Debrecen, Mikszáth Kálmán street",https://www.findahome.hu/ingatlanok/mikszath-k...


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 0 to 850
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Price (HUF)  751 non-null    object
 1   Size         758 non-null    object
 2   Rooms        745 non-null    object
 3   Address      698 non-null    object
 4   Link         760 non-null    object
dtypes: object(5)
memory usage: 35.6+ KB


In [128]:
df_droppedna = df.loc[(df['Price (HUF)'].notna()) & (df['Rooms'].notna())]
df_droppedna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 739 entries, 0 to 850
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Price (HUF)  739 non-null    object
 1   Size         739 non-null    object
 2   Rooms        739 non-null    object
 3   Address      696 non-null    object
 4   Link         739 non-null    object
dtypes: object(5)
memory usage: 34.6+ KB


In [129]:
df_droppedna['Price (HUF)'] = df_droppedna['Price (HUF)'].str.replace(' ', '').str.replace('\xa0', '').str.strip().astype(int)
df_droppedna['Size'] = df_droppedna['Size'].str.replace(',', '.').str.replace('m2', '').str.strip()
df_droppedna['Size'].replace(['-', '?', ''], np.nan, inplace=True)
df_droppedna['Size'] = df_droppedna['Size'].astype(float).round(1)
df_droppedna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 739 entries, 0 to 850
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Price (HUF)  739 non-null    int32  
 1   Size         715 non-null    float64
 2   Rooms        739 non-null    object 
 3   Address      696 non-null    object 
 4   Link         739 non-null    object 
dtypes: float64(1), int32(1), object(3)
memory usage: 31.8+ KB


C:\Users\youse\AppData\Local\Temp\ipykernel_1784\1029337834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_droppedna['Price (HUF)'] = df_droppedna['Price (HUF)'].str.replace(' ', '').str.replace('\xa0', '').str.strip().astype(int)
C:\Users\youse\AppData\Local\Temp\ipykernel_1784\1029337834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_droppedna['Size'] = df_droppedna['Size'].str.replace(',', '.').str.replace('m2', '').str.strip()
C:\Users\youse\AppData\Local\Temp\ipykernel_1784\1029337834.p

In [130]:
df_findahome = df_droppedna.loc[df_droppedna['Link'].str.contains('findahome')]
df_great = df_droppedna.loc[(df_droppedna['Link'].str.contains('greatforest')) & (df_droppedna['Link'].str.contains('findahome') == False)]
df_rent = df_droppedna.loc[df_droppedna['Link'].str.contains('debrecenrent.hu')]

In [131]:
df_findahome

,Price (HUF),Size,Rooms,Address,Link
0,120000,35.0,"1 living room(s), 1 bedroom(s), 1 bathroom(s)","Debrecen, Sinai Miklós",https://www.findahome.hu/ingatlanok/debrecen-s...
1,140000,59.0,"1 living room(s), 1 bedroom(s), 1 bathroom(s)","Debrecen, Kardos Albert",https://www.findahome.hu/ingatlanok/debrecen-k...
2,280000,100.0,"1 living room(s), 4 bedroom(s), 1 bathroom(s)","Debrecen, Komlóssy street",https://www.findahome.hu/ingatlanok/debrecen1-...
3,420000,112.0,"1 living room(s), 3 bedroom(s), 1 bathroom(s)","Debrecen, Rákóczi",https://www.findahome.hu/ingatlanok/debrecen-b...
4,180000,56.0,"0 living room(s), 2 bedroom(s), 1 bathroom(s)","Debrecen, Mikszáth Kálmán street",https://www.findahome.hu/ingatlanok/mikszath-k...
...,...,...,...,...,...
648,90000,55.0,"1 living room(s), 1 bedroom(s), 1 bathroom(s)","Debrecen, Poroszlay street",https://www.findahome.hu/ingatlanok/2-bedrooms...
649,150000,80.0,"1 living room(s), 3 bedroom(s), 1 bathroom(s)","Debrecen, Honvéd",https://www.findahome.hu/ingatlanok/flat-for-r...
650,80000,54.0,"1 living room(s), 1 bedroom(s), 1 bathroom(s)","Debrecen, Doberdó street",https://www.findahome.hu/ingatlanok/2-rooms-fl...
651,160000,43.0,"0 living room(s), 1 bedroom(s), 1 bathroom(s)","Debrecen, Dóczy Residential Area",https://www.findahome.hu/ingatlanok/doczy-resi...


In [132]:
df3 = df_findahome['Rooms'].str.split(',', expand=True)
df3.columns = ['Rooms{}'.format(x+1) for x in df3.columns]
# df_findahome[['Living_rooms', 'Bedrooms', 'Bathrooms']] = df_findahome.Rooms.str.split(',', expand=True)
df_findahome['Living_rooms'] = df3['Rooms1'].str[:-14].str.strip()
df_findahome['Living_rooms'] = df_findahome['Living_rooms'].astype(int)
df_findahome['Bedrooms'] = df3['Rooms2'].str[:-10].str.strip()
df_findahome['Bedrooms'] = df_findahome['Bedrooms'].astype(int)
df_findahome['Bathrooms'] = df3['Rooms3'].str[:-11].str.strip()
df_findahome['Bathrooms'] = df_findahome['Bathrooms'].astype(int)
df_findahome['Rooms'] = df_findahome['Living_rooms'] + df_findahome['Bedrooms']
df_findahome['Rooms'] = df_findahome['Rooms'].astype(int)
df_findahome.drop(columns=['Living_rooms', 'Bedrooms'], inplace=True)

df3 = df_rent['Rooms'].str.split(',', expand=True)
df3.columns = ['Rooms{}'.format(x+1) for x in df3.columns]
df_rent['Bedrooms'] = df3['Rooms1'].str[:-8].str.strip()
df_rent['Bedrooms'] = df_rent['Bedrooms'].astype(int)
df_rent['Bathrooms'] = df3['Rooms2'].str[:-9].str.strip()
df_rent['Bathrooms'] = df_rent['Bathrooms'].astype(int)
df_rent['Rooms'] = df_rent['Bedrooms']
df_rent['Rooms'] = df_rent['Rooms'].astype(int)
df_rent.drop(columns=['Bedrooms'], inplace=True)

df_great['Rooms'] = df_great['Rooms'].astype(int)
dfs = [df_findahome, df_great, df_rent]
df1 = pd.concat(dfs)
df1 = df1[['Price (HUF)', 'Size', 'Rooms', 'Address', 'Bathrooms', 'Link']]
df1['Size'] = df1['Size'].astype(float).round(1)
df1['Bathrooms'].fillna(0)
df1['Bathrooms'] = df1['Bathrooms'].astype(float).round(0)

C:\Users\youse\AppData\Local\Temp\ipykernel_1784\3801851828.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_findahome['Living_rooms'] = df3['Rooms1'].str[:-14].str.strip()
C:\Users\youse\AppData\Local\Temp\ipykernel_1784\3801851828.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_findahome['Living_rooms'] = df_findahome['Living_rooms'].astype(int)
C:\Users\youse\AppData\Local\Temp\ipykernel_1784\3801851828.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [133]:
df1.rename(columns={'Size':'Size (m2)'}, inplace=True)
df1 = df1[df1['Price (HUF)'] < 1_000_000]
df1.to_excel(r'C:\Users\youse\Desktop\Data Analysis Portfolio\Python\Data Scraping\streamlit\Houses_Cleaned.xlsx', index=False)
print(len(df1))

719
